## Import


In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import requests
from io import StringIO
import os
import pyarrow.parquet as pq

## Setup

In [2]:
# # Connection string
# connection_string = 'sqlite:///your_database.db'  # Replace with your actual connection string
# 
# # Create SQLAlchemy engine
# engine = create_engine(connection_string)

### Connecting to Mysql DB


In [3]:
DATABASE_URL = "mysql+pymysql://root:a@localhost/p_taxi"
engine = create_engine(DATABASE_URL)

### Create Table

In [6]:
with engine.connect() as connection:
    with open('create_table.sql', 'r') as file:
        create_table_query = file.read()
    connection.execute(text(create_table_query))

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CREATE TABLE IF NOT EXISTS location_table (\n    locationID INTEGER AUTO_INCREMEN' at line 26")
[SQL: CREATE TABLE IF NOT EXISTS raw_table (
    tripID INTEGER AUTO_INCREMENT PRIMARY KEY,
    VendorID INTEGER,
    create_time DATETIME DEFAULT CURRENT_TIMESTAMP,
    tpep_pickup_datetime DATETIME,
    tpep_dropoff_datetime DATETIME,
    passenger_count INTEGER,
    trip_distance FLOAT,
    RateCodeID INTEGER,
    store_and_fwd_flag TEXT,
    payment_type INTEGER,
    PULocationID INTEGER,
    DOLocationID INTEGER,
    fare_amount FLOAT,
    extra FLOAT,
    mta_tax FLOAT,
    tip_amount FLOAT,
    tolls_amount FLOAT,
    improvement_surcharge FLOAT,
    total_amount FLOAT,
    trip_duration_min FLOAT,
    congestion_surcharge FLOAT,
    Airport_fee FLOAT
);

CREATE TABLE IF NOT EXISTS location_table

# !CLEARS TABLE!

In [5]:
with engine.connect() as connection:
    with open('empty_table.sql', 'r') as file:
        create_table_query = file.read()
    connection.execute(text(create_table_query))

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'DROP TABLE location_table' at line 2")
[SQL: DROP TABLE raw_table;
DROP TABLE location_table;]
(Background on this error at: https://sqlalche.me/e/14/f405)

### Load .parquet into a DataFrame

In [13]:
# Step 3: Read the Parquet file using pyarrow
trips = pq.read_table('csvs/yellow_tripdata_2024-05.parquet')
df = trips.to_pandas()

print(df.head())
# # Step 4: Insert data into the table
df.to_sql('staging', con=engine, if_exists='append', index=False)

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2024-05-01 00:59:15   2024-05-01 01:23:50              1.0   
1         2  2024-04-30 23:58:26   2024-05-01 00:29:42              1.0   
2         2  2024-05-01 00:57:17   2024-05-01 01:14:15              1.0   
3         2  2024-05-01 00:24:47   2024-05-01 00:48:51              1.0   
4         2  2024-05-01 00:11:20   2024-05-01 00:52:10              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           6.10         1.0                  N           138           145   
1          11.23         1.0                  N           138           249   
2           9.02         1.0                  N           138           170   
3           6.53         1.0                  N            87           133   
4          14.38         1.0                  N           161           165   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


3723833

### Load .csv into a DataFrame  

In [10]:
location_csv = 'csvs/taxi_zone_lookup.csv'
df_location = pd.read_csv(location_csv)

df_location.to_sql('location_table', con=engine, if_exists='append', index=False)

265

### Run ETL

In [ ]:
with engine.connect() as connection:
    with open('etl.sql', 'r') as file:
        etl_query = file.read()
    connection.execute(text(etl_query))